# Scheduling operations
In this notebook we will cover the QGym `Scheduling` environment.

This environment is aimed at solving the problem of scheduling operations in the best possibly way, whilst taking hardware constraints and commutation rules into account.